In [1]:
# if not available, run this cell to install requiresd packages
! pip install nltk
! pip install numpy
! pip install pandas
! pip install matplotlib



^C
ERROR: Operation cancelled by user


In [3]:
# import required packages
import re
import nltk
import string

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from nltk.sentiment.vader import SentimentIntensityAnalyzer


KeyboardInterrupt: 

In [ ]:
# inplace the location of the file on your machine
file_path = "/Users/eugenernst/Desktop/C-SCM-DATA-Candidates_Evaluation_Anonymized_SS21.xlsx"

In [ ]:
print(file_path)

In [ ]:
# download of specific tools for nltk (you have to do it once)
nltk.download('vader_lexicon')
nltk.download("stopwords")

stopwords = nltk.corpus.stopwords.words("english")

In [ ]:
def evaluator_count(data=None):
    if data is None:
        data = pd.read_excel(file_path, engine='openpyxl')
    eval_count = data.groupby('Evaluated by Code').count()[['Person ID', 'Agreed By Code']]

    eval_count.columns = ['evaluated', 'agreed']
    return eval_count

#
eval_count = evaluator_count()

# gives the evaluator (index), number of evaluations and count of evaluations which have been agreed by other persons
eval_count.head(10)


In [ ]:
# checks whether number is included
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

# filters numbers (ids) from the statements
def filter_numbers(word_list):
    return [word for word in word_list if hasNumbers(word) is False]

def filter_stop_words(word_list):
    return [word for word in word_list if word.lower() not in stopwords]


def prepare_word_list():
    data = pd.read_excel(file_path, engine='openpyxl')
    statements = data['Evaluation Statement'].tolist()

    word_list = []
    for statement in statements:
        #statement = statements[0]
        if type(statement) == str:
            # remove punctuation marks
            statement = statement.translate(str.maketrans({a:None for a in string.punctuation}))
            # split statement into single words
            statement_words = nltk.word_tokenize(statement)
            # remove numeric values
            statement_words = filter_numbers(statement_words)
            # remove stopwords
            statement_words = filter_stop_words(statement_words)

            word_list += statement_words

    return word_list


def worde_freq_analysis(num_top=None):
    word_list = prepare_word_list()

    if num_top is None:
        num_top = len(np.unique(word_list))

    fd = nltk.FreqDist(word_list)

    most_common_words = pd.DataFrame(fd.most_common(num_top), columns=['word', 'count'])

    return most_common_words


word_ranking = worde_freq_analysis()

# top 20 words and the count of words
word_ranking.head(20)

In [ ]:
def statement_sentiment(statement):
    vader = SentimentIntensityAnalyzer()
    polarity_dict = vader.polarity_scores(statement)
    return polarity_dict

def sentiment_data():
    data = pd.read_excel(file_path, engine='openpyxl')
    data.dropna(axis=0, inplace=True)
    data['Person ID'] = data['Person ID'].astype(int)
    data.set_index('Person ID', inplace=True)

    sentiments = ['neg', 'neu', 'pos', 'compound']

    for sentiment in sentiments:
        data[sentiment] = 0

    sentiment_index_dict = {}
    for index in data.index:
        statement = data.loc[index]['Evaluation Statement']
        try:
            sentiment_index_dict[index] = statement_sentiment(statement)

        except:
            print(statement)

        for sentiment in sentiments:
            data.loc[index, sentiment] = np.copy(sentiment_index_dict[index][sentiment])

    return data


data = sentiment_data()

sentiment_columns = ['neg', 'neu', 'pos', 'compound']

senti = data[sentiment_columns].head(10)
